In [5]:
# Heston Monte Carlo basket option pricer. Adapted from:
# https://hal.sorbonne-universite.fr/hal-02273889/document

import numpy as np
import time, os
import pandas as pd
import datetime
np.set_printoptions(edgeitems=9, linewidth=128,
                    formatter={'float':lambda x: f'{x:.3f}'}, precision=3, suppress=True)
# pd.options.display.float_format = '{:.3f}'.format

# Each simulation prices num_mats contracts of equally spaced maturities and num_strikes contracts
# for strikes of moneyness 0.5-1.5 for each maturity.

# Seed values
#dim        = 3                             # Dimension of basket (number of stocks)
num_mats    = 20                            # Number of maturities considered per simulation
mc_steps    = 100*num_mats                  # Shorter maturities are sampled at intervals after every 100 mc steps.
num_strikes = 40                            # Number of strike prices considered per simulation
num_samples = 50                            # Output has length num_samples*num_mats*num_strikes.

# Sample rates & spots from real-world frequencies, without excessive outliers
real_data = pd.read_csv('calls_OMrates587569.csv')
mask = (  (real_data.underlyings_price > 0.1   )
        & (real_data.underlyings_price < 290   )
        & (real_data.rate              < 0.055 )  )
real_data = real_data[mask]

# Corr is dim-vector of correlations between stock price and variance processes.
# S0 is dim-vector of initial stock prices.
def mc_heston(dim, S0, K, T, initial_var, long_term_var, rate_reversion,
              vol_of_vol, corr, r, num_reps, mc_steps, ntime, nK):
    option_type = 'c'
    eps         = 0.0001
    T          /= 365
    delta_t     = T / float(mc_steps)
    Vt          = np.zeros([ int(mc_steps) ])
    W1          = np.zeros([ int(mc_steps) ])
    corr2       = np.sqrt(1 - corr ** 2)
    aux1        = 0.5 * vol_of_vol * np.sqrt(delta_t)
    aux2        = 0.25 * vol_of_vol ** 2 * delta_t
    payoff      = np.zeros([int(mc_steps / ntime), nK])
    for i in range(num_reps):
        vt = initial_var
        for j in range(mc_steps):
            W1[j] = np.random.normal(0, 1)
            vt = (np.sqrt(vt) + aux1 * W1[j]) ** 2 / - rate_reversion * (vt - long_term_var) * delta_t - aux2
            vt=max(eps,vt)
            Vt[j] = vt
        st = np.log(S0)
        for j in range(mc_steps):
            aux3 = (r - 0.5 * Vt[j]) * delta_t
            aux4 = np.sqrt(Vt[j] * delta_t)
            for k in range(dim):
                w2=(corr[k] * W1[j] + corr2[k] * np.random.normal(0, 1))
                st[k] += aux3 + aux4 * w2
            jj = int((j+1) / ntime)                              # mc steps in units of ntime, floored
            if jj * ntime == j + 1:                              # Completed j+1 mc steps. Check if a non-zero multiple of ntime
                sst = np.sum(np.exp(st))
                strikies = np.linspace(K, K/3, nK)               # K=2*sum(spot), 0.5 < Moneyness < 1.5, 2sum/3 < k < 3sum/2
                for k in range(nK):                              # Evaluate maxK first, but still in last column
                    if option_type == 'c':
                        payoff[jj-1, -1-k] += max(sst - strikies[k], 0) # -ve index so nK=1 uses k=K not k=K/num_strikes
                    elif option_type == 'p':
                        payoff[jj-1, k-1] += max(strikies[k] - sst, 0)
    maturities = np.linspace(T / num_mats, T, num_mats)[:, None]
    return (payoff / float(num_reps)) * (np.exp(-r * maturities))

def rand(num = num_samples):
    return np.random.random(num)

def corr(dim):
    return -0.05 - 0.7 * rand(dim) # 

def spots(dim): # Initial spot price of each sampled from historical data
    return np.array(real_data['underlyings_price'].sample(n=dim, replace=True))

def generate_inputs(dim):                      
    maturity       = np.array(real_data['days_to_maturity'].sample(n = num_samples, replace = True))
    rate           = np.array(real_data['rate'].sample(n = num_samples, replace = True))
    rate_reversion = 0.01 + 5 * rand()             # kappa
    vol_of_vol     = 0.01 + 0.7 * rand()           # lambda
    long_term_var  = 0.001 + 0.05 * rand()         # theta
    initial_var    = 0.001 + 0.05 * rand()         # sqrt(v0)
    dims           = np.repeat(dim, num_samples)
    num_step       = np.repeat(mc_steps, num_samples)
    num_rep        = np.repeat(15, num_samples) # Convergence
    nmat           = np.repeat(int(mc_steps / num_mats), num_samples) # Monte Carlo steps per maturity
    num_strike     = np.repeat(num_strikes, num_samples)
    inputs         = np.array([dims, np.round(maturity), initial_var, long_term_var, rate_reversion,
                               vol_of_vol, rate, num_rep, num_step, nmat, num_strike]).T.tolist()
    strikes        = []
    for i in range(len(inputs)):
        spot = spots(dim)
        strikes.append(sum(spot) * 3 / 2)         # Max strike will have 1.5 moneyness, min 1.5/3=0.5
        inputs[i].insert(1, spot)
        inputs[i].insert(2, sum(spot) * 3 / 2)
        inputs[i].insert(8, corr(dim))

    inputs     = np.array(inputs, dtype = object)
    strikes    = np.array(strikes)
    strikes    = np.linspace(strikes / 3, strikes, num_strikes).repeat(num_mats, 1).reshape(num_strikes, num_samples, num_mats).transpose(1,2,0)
    maturities = np.linspace(maturity / num_mats, maturity, num_mats).repeat(num_strikes, 1).reshape(num_mats, num_samples, num_strikes).transpose(1,0,2)
    return inputs, strikes, maturities

def many_hestons(df, ss, ms, dim):
    results = []
    for i in range(len(df)): # Outputs prices for each maturity (row) and strike (col)
        results.append(mc_heston(*df[i]))
        
    # Associate prices with their maturity and strike
    delKT   = np.delete(df, np.s_[2:4], axis = 1)
    sizeup  = np.repeat(delKT[:, None, :], num_mats * num_strikes, 1).reshape(num_samples, num_mats, num_strikes, 12)
    triples = np.stack((ss, ms, np.array(results)), axis=3)
    arr     = np.concatenate((sizeup, triples), axis=3).reshape(num_samples * num_mats * num_strikes, -1)
    
    # Reshape results partitioned by strikes and maturities into a 2D array with spot+corr inner-arrays flattened..
    results                = np.empty((len(arr), 13 + 2*dim))         # Efficiently create dummy array of correct size
    results[:, :1]          = arr[:, :1]                              # LHS float in first position
    results[:, 1+dim:5+dim] = arr[:, 2:6]                             # Middle 4 floats between array positions
    results[:, -8:]         = arr[:, -8:]                             # RHS 8 floats on RHS
    # Slicing at a single index converts each inner-array to floats
    results[:, 1:1+dim]       = np.vstack(arr[:, 1])
    results[:, 5+dim:5+2*dim] = np.vstack(arr[:, -9])
    results                   = np.delete(results, np.s_[(0,-4,-5,-6,-7)], axis = 1)
    return results

In [26]:
dim = 2
maturity       = np.array(real_data['days_to_maturity'].sample(n = num_samples, replace = True))
rate           = np.array(real_data['rate'].sample(n = num_samples, replace = True))
rate_reversion = 0.01 + 5 * rand()             # kappa
vol_of_vol     = 0.01 + 0.7 * rand()           # lambda
long_term_var  = 0.001 + 0.05 * rand()         # theta
initial_var    = 0.001 + 0.05 * rand()         # sqrt(v0)
dims           = np.repeat(dim, num_samples)
num_step       = np.repeat(mc_steps, num_samples)
num_rep        = np.repeat(15, num_samples) # Convergence
nmat           = np.repeat(int(mc_steps/num_mats), num_samples) # Monte Carlo steps per maturity
num_strike     = np.repeat(num_strikes, num_samples)
inputs         = np.array([dims, np.round(maturity), initial_var, long_term_var, rate_reversion,
                  vol_of_vol, rate, num_rep, num_step, nmat, num_strike]).T.tolist()

In [ ]:
folder = '.\\heston_prices'  + datetime.datetime.now().strftime("%d-%H%M") + '\\'
os.mkdir(folder)

# Using time.time rather than timeit as execution on order of
# hundreds of seconds and convenience for retaining output.
times_list = []
def generate_heston_files(dim):
    print('----------------------')
    print(f'Dimension: {dim}/20')
    start = time.time()

    df, ss, ms = generate_inputs(dim)
    time1 = time.time()

    results = many_hestons(df, ss, ms, dim)
    time2 = time.time()
    
    np.savetxt(folder + f'heston_prices_dim{dim:02}.csv', results, delimiter=',')
    time3 = time.time()
    
    times_list.append([dim, len(results[0])-1, time1-start, time2-time1, time3-time2, time.time()-start])

    print(f'Total Time Taken: {time.time()-start}')

In [ ]:
for d in [1, 4, 7, 10, 13, 16]:
    generate_heston_files(d)

In [ ]:
times = pd.DataFrame(times_list, columns=['Dimension', 'Feature Vector Length',
                                          'Generatng Inputs Time', 'Heston Pricing Time',
                                          'Saving Time', 'Total Time Taken'])
times.to_csv(folder + f'heston_times.csv', index=False)

In [ ]:
display(times)

In [ ]:
time = times[times.columns[-1]].sum()
print(f'Overall Time Taken: {time // 3600} Hours, {(time % 3600) // 60} Minutes, {(time % (3600 * 60)) // 60:00}')